**Step 0**: Convert the pr.data.0.Current to csv file first and load both pr.data.0.Current.csv and population.json into two different dataframes

In [10]:
import pandas as pd
import boto3
from io import StringIO

# Initialize S3 client
s3 = boto3.client('s3')

# Data sources
S3_BUCKET_NAME = "rearc-thirdparty-datahub-us-east-2"
S3_KEY = "bls/pr.data.0.Current"
CSV_OUTPUT_KEY = "bls/pr.data.0.Current.csv"

# Read data from S3
response = s3.get_object(Bucket=S3_BUCKET_NAME, Key=S3_KEY)
data = response['Body'].read().decode('utf-8')

# Convert to DataFrame
df = pd.read_csv(StringIO(data), sep='\t')

# Save as CSV
csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# Upload CSV to S3
s3.put_object(Bucket=S3_BUCKET_NAME, Key=CSV_OUTPUT_KEY, Body=csv_buffer.getvalue())



{'ResponseMetadata': {'RequestId': 'T78HJJ3295HSFZZC',
  'HostId': 'zS7Z5dn9tbk3gO6qtR6MGYzLrRNl67D62YcQrGGtuEgyXzJV1CnLm3RtDpKZCiNPeWvS6ZWBASM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'zS7Z5dn9tbk3gO6qtR6MGYzLrRNl67D62YcQrGGtuEgyXzJV1CnLm3RtDpKZCiNPeWvS6ZWBASM=',
   'x-amz-request-id': 'T78HJJ3295HSFZZC',
   'date': 'Thu, 02 May 2024 02:57:23 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"453da51e34469c39b5d8b130e3d4b5e9"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"453da51e34469c39b5d8b130e3d4b5e9"',
 'ServerSideEncryption': 'AES256'}

In [21]:
import pandas as pd
import boto3
from io import StringIO
import json

# Initialize S3 client
s3 = boto3.client('s3')

# Define function to read CSV file from S3
def read_csv_from_s3(bucket_name, key):
    response = s3.get_object(Bucket=bucket_name, Key=key)
    csv_content = response['Body'].read().decode('utf-8')
    return pd.read_csv(StringIO(csv_content))

# Define function to read JSON file from S3
def read_json_from_s3(bucket_name, key):
    response = s3.get_object(Bucket=bucket_name, Key=key)
    json_content = response['Body'].read().decode('utf-8')
    return pd.json_normalize(json.loads(json_content)['data'])

# S3 bucket and file paths
S3_BUCKET_NAME = "rearc-thirdparty-datahub-us-east-2"
CURRENT_DATA = "bls/pr.data.0.Current.csv"
POPULATION_DATA = "population/population.json"

# Load data from S3
current = read_csv_from_s3(S3_BUCKET_NAME, CURRENT_DATA)
population = read_json_from_s3(S3_BUCKET_NAME, POPULATION_DATA)

print(current)
print(population)

       series_id          year period         value footnote_codes
0      PRS30006011        1995    Q01         2.600            NaN
1      PRS30006011        1995    Q02         2.100            NaN
2      PRS30006011        1995    Q03         0.900            NaN
3      PRS30006011        1995    Q04         0.100            NaN
4      PRS30006011        1995    Q05         1.400            NaN
...                  ...   ...    ...           ...            ...
35461  PRS88003203        2022    Q04       115.039            NaN
35462  PRS88003203        2022    Q05       113.713            NaN
35463  PRS88003203        2023    Q01       116.214            NaN
35464  PRS88003203        2023    Q02       116.953            NaN
35465  PRS88003203        2023    Q03       116.928              R

[35466 rows x 5 columns]
  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2021  2021   329725481  united-states
1   01000US  United States     2

**Step 1**: Using the dataframe from the population data API (Part 2), generate the mean and the standard deviation of the US population across the years [2013, 2018] inclusive.


In [29]:
# Filter data between [2013, 2018] inclusive of US population
us_population_stats = population[(population["Year"].astype(int) >= 2013) &
                                  (population["Year"].astype(int) <= 2018)]

# Calculating mean and standard deviation of population
us_population_stats_mean = us_population_stats["Population"].mean()
us_population_stats_std = us_population_stats["Population"].std()

(us_population_stats_mean, us_population_stats_std)


(317437383.0, 4257089.5415293295)

**Step 2**: Using the dataframe from the time-series (Part 1), For every series_id, find the best year: the year with the max/largest sum of "value" for all quarters in that year. Generate a report with each series id, the best year for that series, and the summed value for that year.

In [31]:
# Remove columns with leading and trailing spaces
current.rename(columns={"series_id        ": "series_id"}, inplace=True)
current.rename(columns={"       value": "value"}, inplace=True)

# Generate a report with series_id, best_year, and aggregate value
top_value_series = current.groupby(["series_id", "year"], as_index=False)["value"].agg("sum")
top_value_series = top_value_series.sort_values("value", ascending=False).drop_duplicates("series_id", keep="first").sort_index().reset_index(drop=True)
top_value_series


,series_id,year,value
0,PRS30006011,2022,20.500
1,PRS30006012,2022,17.100
2,PRS30006013,1998,704.125
3,PRS30006021,2010,17.600
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,282.800
278,PRS88003193,2022,765.817
279,PRS88003201,2022,36.100
280,PRS88003202,2022,28.900


**Step 3**: Using both dataframes from Part 1 and Part 2, generate a report that will provide the value for series_id = PRS30006032 and period = Q01 and the population for that given year (if available in the population dataset)

In [35]:
# Filter current data with series_id = PRS30006032 and period = Q01
filtered_series = current.loc[current["series_id"].str.contains("PRS30006032", case=False)]
filtered_series = filtered_series[filtered_series["period"].str.contains("Q01", case=False)]

# Filter population data for the same year
population_extract = population["Population"].astype(int)
population["Year"] = population["Year"].astype(int)

# Merge and filter results
result = pd.merge(filtered_series, population, left_on="year", right_on="Year", how="left")
result[["series_id", "year", "period", "value", "Population"]]


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.4,NaN
2,PRS30006032,1997,Q01,2.7,NaN
3,PRS30006032,1998,Q01,1.0,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.5,NaN
7,PRS30006032,2002,Q01,-6.7,NaN
8,PRS30006032,2003,Q01,-5.6,NaN
9,PRS30006032,2004,Q01,2.1,NaN
